In [ ]:
from bs4 import BeautifulSoup
import requests

import multiprocessing as mp
import concurrent.futures

import time
import re
import os
import pandas as pd

In [ ]:
# Parsing business world news article with beautifulsoup + multithreading
# NOTE: Don't forget to modify this one such that if pages > 900, divide it by 2 or 3 to parse by batches

def _parse_content(url):
    headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = BeautifulSoup(requests.get(url, headers).content, 
                            'html.parser')
    return content

def _get_article_b4(inputParam):
    """helper function"""
    dir_output = r'C:\Users\ojell\Desktop\Oj\_Thesis\Data\news\businessworld'
    url = inputParam[0]
    category = inputParam[1]
    lstpage = inputParam[2]
    ArtUrl = []
    for pg in range(1, lstpage):
        content = _parse_content(url + f"page/{str(pg)}/")
        page = content.find('div', {'class': 'td-container td-category-container'})
        next10 = page.find('div', {'class': 'td-ss-main-content'}).find_all('a', {'class': 'td-image-wrap'})
        if pg % 50 == 0: time.sleep(10)
        if pg == 1:
            top5 = page.find('div', {'class': 'td-big-grid-wrapper'}).find_all('a', {'class': 'td-image-wrap'})
            ArtUrl = ArtUrl + [item.get('href') for item in top5] + [item.get('href') for item in next10]
        else:
            ArtUrl = ArtUrl + [item.get('href') for item in next10]
        dct = {"Article Url": ArtUrl, "Category": category}
    df = pd.DataFrame.from_dict(dct)
    filename = category.lower() + '.csv'
    df.to_csv(os.path.join(dir_output, filename), index=False)
    return df

def par_main(news_url):
    exclude = ['buying rates', 'foreign interest rates', 'philippine mutual funds',
               'leaders and laggards', 'stock quotes', 'stock markets summary',
               'non-bsp convertible currencies', 'bsp convertible currencies', 'us commodity futures',
               'health',
               'top stories', 'corporate','stock market', 'banking', 'economy',
               'markets', 'the nation', 'world', 'opinion', 'infographics', 
               'b-side podcasts', 'sparkup', 'spotlight', 'labor', 'property', 
               'agribusiness', 'arts & leisure', 'technology', 'velocity',
               'special features', 'sports', 'special reports']
    
    content = _parse_content(news_url)

    catContent = content.find('ul', {'id': 'menu-header-menu-1'}).find_all('a')
    category = [item.text.title() for item in catContent if item.text.lower() not in exclude]
    category_url = [item.get('href') for item in catContent if item.text.lower() not in exclude]
    
    pagetab = [_parse_content(url).find('span', {'class': 'pages'}).text for url in category_url]
    lstpg = [int(re.sub(r'[^\w\s]', '', page[-(len(page) - 10):])) for page in pagetab]
    
    output = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
        futures = [pool.submit(_get_article_b4, [category_url[i], category[i], lstpg[i]]) for i in range(0, 1)]
        for f in concurrent.futures.as_completed(futures):
            output += 1
    return category

newsWeb = 'https://www.bworldonline.com/'

In [ ]:
start = time.time()
test = par_main(newsWeb)
par_time = time.time() - start
#print(f"Parallel Output: {len(test)}")
print(f"Parallel elapsed time: {round(par_time/60, 2)} mins")

### 